## <font color='blue'> Residual Networks </font>


This notebook is based on the programming assignment of deeplearning.ai, course Convolutional Neural Networks, week Deep convolutional models case studies. The figures are also taken from that assignment.

Very deep networks can represent very complex functions; but in practice, they are hard to train. Residual Networks, introduced by [He et al.](https://arxiv.org/pdf/1512.03385.pdf), allow us to train much deeper networks than were previously practically feasible.

## Learning objectives

- Application of Convolutional Neural Networks in TensorFlow
- Apply Batch Normalization
- Use of Adam optimizer 
- Use of GPU for the training of the model

In [2]:
import numpy as np
#import pydot
from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import time
import math
import h5py
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
print(tf.__version__)

config = tf.ConfigProto()
config.gpu_options.allow_growth=True



1.14.0


In [3]:
import warnings

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [4]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [5]:
def build_inputs():
    
    inputs_ = tf.placeholder(tf.float32,[None,64,64,3],name='inputs_')
    targets_ = tf.placeholder(tf.float32,[None,6], name='targets_')
    training = tf.placeholder_with_default(False, shape=(), name='training')
    
    return inputs_, targets_, training

In [6]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

## Building a Residual Network

In ResNets, a "shortcut" or a "skip connection" allows the gradient to be directly backpropagated to earlier layers:  

<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'> </u><font color='purple'> A ResNet block showing a **skip-connection** <br> </center></caption>

### The identity block

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> </u><font color='purple'> **Identity block.** Skip connection "skips over" 3 layers.</center></caption>

In [7]:
def identity_block(X, is_training, f, filters, stage, block):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'    
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    with tf.variable_scope('W1_identity'):
        strides = 1
        X = tf.layers.conv2d(X, F1, kernel_size = [1, 1], strides = [strides, strides],
                             padding='VALID',kernel_initializer = tf.glorot_uniform_initializer(seed = 0),
                             name = conv_name_base + '2a')
        X = tf.layers.batch_normalization(X, training=is_training, name = bn_name_base + '2a')
        X = tf.nn.relu(X)
    
    # Second component of main path
    with tf.variable_scope('W2_identity'):
        strides = 1
        X = tf.layers.conv2d(X, F2, kernel_size = [f, f], strides = [strides, strides],
                             padding='SAME',kernel_initializer = tf.glorot_uniform_initializer(seed = 0),
                             name = conv_name_base + '2b')
        X = tf.layers.batch_normalization(X, training=is_training, name = bn_name_base + '2b')
        X = tf.nn.relu(X)
    
    # Third component of main path
    with tf.variable_scope('W3_identity'):
        
        strides = 1
        X = tf.layers.conv2d(X, F3, kernel_size = [1, 1], strides = [strides, strides],
                             padding='VALID',kernel_initializer = tf.glorot_uniform_initializer(seed = 0),
                             name = conv_name_base + '2c')
        X = tf.layers.batch_normalization(X, training=is_training, name = bn_name_base + '2c')
    
    #Final step: Add shortcut value to main path, and pass it through a RELU activation
        X=tf.add(X,X_shortcut)
        X = tf.nn.relu(X)
    
    return X

### The convolutional block

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> </u><font color='purple'>  **Convolutional block** </center></caption>

In [8]:
def convolutional_block(X, is_training, f, filters, stage, block, s = 2):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path 
    with tf.variable_scope('W1_convolutional'):
        strides = s
        X = tf.layers.conv2d(X, F1, kernel_size = [1, 1], strides = [strides, strides],
                             padding='VALID',kernel_initializer = tf.glorot_uniform_initializer(seed = 0),
                             name = conv_name_base + '2a')
        X = tf.layers.batch_normalization(X, training=is_training, name = bn_name_base + '2a')
        X = tf.nn.relu(X)

    # Second component of main path (≈3 lines)
    with tf.variable_scope('W2_convolutional'):
        strides = 1
        X = tf.layers.conv2d(X, F2, kernel_size = [f, f], strides = [strides, strides],
                             padding='SAME',kernel_initializer = tf.glorot_uniform_initializer(seed = 0),
                             name = conv_name_base + '2b')
        X = tf.layers.batch_normalization(X, training=is_training, name = bn_name_base + '2b')
        X = tf.nn.relu(X)
    

    # Third component of main path (≈2 lines)
    with tf.variable_scope('W3_convolutional'):
        strides = 1
        X = tf.layers.conv2d(X, F3, kernel_size = [1, 1], strides = [strides, strides],
                             padding='VALID',kernel_initializer = tf.glorot_uniform_initializer(seed = 0),
                             name = conv_name_base + '2c')
        X = tf.layers.batch_normalization(X, training=is_training, name = bn_name_base + '2c')

    ##### SHORTCUT PATH #### (≈2 lines)
    with tf.variable_scope('Wshort_convolutional'):
        strides = s
        X_shortcut = tf.layers.conv2d(X_shortcut, F3, kernel_size = [1, 1], strides = [strides, strides],
                             padding='VALID',kernel_initializer = tf.glorot_uniform_initializer(seed = 0),
                             name = conv_name_base + '1')
        X_shortcut = tf.layers.batch_normalization(X_shortcut, training=is_training, name = bn_name_base + '1')

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
        X=tf.add(X,X_shortcut)
        X = tf.nn.relu(X)
    
    ### END CODE HERE ###
    
    return X

In [9]:
def build_loss(logits, targets):
        
    # Softmax cross entropy loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets))
    
    return loss

In [10]:
def build_optimizer(loss, learning_rate):
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    return optimizer

## Building the ResNet model (50 layers)

The following figure describes in detail the architecture of this neural network. "ID BLOCK" in the diagram stands for "Identity block," and "ID BLOCK x3" means you should stack 3 identity blocks together.

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> </u><font color='purple'>  **ResNet-50 model** </center></caption>

In [11]:
class ResNet_model:
    
    def __init__(self, learning_rate):
    
        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs_, self.targets_, self.training = build_inputs()
        
        # Zero-Padding
        paddings = tf.constant([[0, 0], [3, 3,], [3, 3], [0, 0]])
        self.X = tf.pad(self.inputs_, paddings, "CONSTANT")
        
        # Stage 1
        strides = 2        
        self.X = tf.layers.conv2d(self.X, 64, kernel_size = [7, 7], strides = [strides, strides],
                             padding='VALID',name = 'conv1')
        self.X = tf.layers.batch_normalization(self.X, training=self.training, name = 'bn_conv1')
        self.X = tf.nn.relu(self.X)
        self.X = tf.nn.max_pool(self.X, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
        
        # Stage 2
        self.X = convolutional_block(self.X, is_training=self.training, f=3, filters = [64, 64, 256], 
                                     stage = 2, block='a', s = 1)
        self.X = identity_block(self.X, is_training=self.training, f=3, filters = [64, 64, 256], 
                                stage = 2, block = 'b')
        self.X = identity_block(self.X, is_training=self.training, f=3, filters = [64, 64, 256], 
                                stage = 2, block = 'c')
        
        # Stage 3
        self.X = convolutional_block(self.X, is_training=self.training, f = 3, filters = [128,128,512], 
                                     stage = 3, block='a',  s = 2)
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [128,128,512], 
                                stage=3, block='b')
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [128,128,512], 
                                stage=3, block='c')
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [128,128,512], 
                                stage=3, block='d')
        
        #Stage 4 
        self.X = convolutional_block(self.X, is_training=self.training, f = 3, filters = [256, 256, 1024], 
                                     stage = 4, block='a', s = 2)
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [256, 256, 1024], 
                                stage=4, block='b')
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [256, 256, 1024], 
                                stage=4, block='c')
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [256, 256, 1024], 
                                stage=4, block='d')
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [256, 256, 1024], 
                                stage=4, block='e')
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [256, 256, 1024], 
                                stage=4, block='f')
        
        # Stage 5 
        self.X = convolutional_block(self.X, is_training=self.training, f = 3, filters = [512, 512, 2048], 
                                     stage = 5, block='a', s = 2)
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [512, 512, 2048], 
                                stage=5, block='b')
        self.X = identity_block(self.X, is_training=self.training, f = 3, filters = [512, 512, 2048], 
                                stage=5, block='c')
        
        # AVGPOOL
        self.X = tf.nn.avg_pool(self.X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID', name='avg_pool')
        
        # output layer
        self.X = tf.contrib.layers.flatten(self.X)
        self.X = tf.contrib.layers.fully_connected(self.X, 6, activation_fn=None, 
                                                   weights_initializer = tf.glorot_uniform_initializer(seed = 0))
        
        self.loss = build_loss(self.X, self.targets_)
        self.optimizer = build_optimizer(self.loss, learning_rate)
        
        correct_prediction = tf.equal(tf.argmax(self.X, 1), tf.argmax(self.targets_, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        self.acc, self.acc_op = tf.metrics.accuracy(labels=tf.argmax(self.targets_, 1), 
                                  predictions=tf.argmax(self.X, 1))

In [12]:
#batch_size = 32
mini_batch_size = 32
learning_rate = 0.001

### Load the Dataset



In [13]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = tf.one_hot(Y_train_orig, depth=6)
Y_test =  tf.one_hot(Y_test_orig, depth=6)
Y_train = tf.reshape(Y_train,[-1,6])
Y_test = tf.reshape(Y_test,[-1,6])

#with tf.Session() as sess:
with tf.Session(config=config) as sess:
    Y_train,Y_test = sess.run([Y_train,Y_test])
    
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [16]:
tf.reset_default_graph()
epochs = 40
model = ResNet_model(learning_rate = learning_rate)
extra_graphkeys_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)



#with tf.Session() as sess:
with tf.Session(config=config) as sess:
    saver = tf.train.Saver() 
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())

    start_time=time.time()
    for e in range(epochs):
        train_acc = []
        iteration = 1
         
        #for (x, y) in get_batches(X_train, Y_train, batch_size):
        for (x, y) in random_mini_batches(X_train, Y_train, mini_batch_size = mini_batch_size, seed = 0):
            feed = {model.inputs_: x,
                   model.targets_: y,
                   model.training: True}

            batch_loss, batch_accuracy, batch_acc, batch_acc_op, _, _= sess.run([model.loss, model.accuracy, 
                                                                                 model.acc, model.acc_op,
                                                      model.optimizer, extra_graphkeys_update_ops],
                                 feed_dict=feed)
            
            train_acc.append(batch_acc_op)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e+1, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(batch_loss),
                      "Train accuracy: {:.4f}".format(batch_acc_op)
                     )
                     
                
            iteration +=1
        print("Train accuracy (mean): {:.4f}".format(np.mean(train_acc)))
        
    duration=time.time()-start_time
    print("duration: {:.1f} sec".format(duration))
    saver.save(sess, "checkpoints/ResNets.ckpt")

Epoch: 1/40 Iteration: 5 Train loss: 5.870 Train accuracy: 0.2313
Epoch: 1/40 Iteration: 10 Train loss: 1.402 Train accuracy: 0.2844
Epoch: 1/40 Iteration: 15 Train loss: 1.335 Train accuracy: 0.3521
Epoch: 1/40 Iteration: 20 Train loss: 1.140 Train accuracy: 0.3734
Epoch: 1/40 Iteration: 25 Train loss: 0.998 Train accuracy: 0.3862
Epoch: 1/40 Iteration: 30 Train loss: 0.882 Train accuracy: 0.4167
Train accuracy (mean): 0.3383
Epoch: 2/40 Iteration: 5 Train loss: 1.074 Train accuracy: 0.4605
Epoch: 2/40 Iteration: 10 Train loss: 0.596 Train accuracy: 0.4907
Epoch: 2/40 Iteration: 15 Train loss: 0.462 Train accuracy: 0.5199
Epoch: 2/40 Iteration: 20 Train loss: 0.512 Train accuracy: 0.5465
Epoch: 2/40 Iteration: 25 Train loss: 0.444 Train accuracy: 0.5649
Epoch: 2/40 Iteration: 30 Train loss: 0.500 Train accuracy: 0.5814
Train accuracy (mean): 0.5269
Epoch: 3/40 Iteration: 5 Train loss: 0.749 Train accuracy: 0.6013
Epoch: 3/40 Iteration: 10 Train loss: 0.512 Train accuracy: 0.6113
Epoch

Train accuracy (mean): 0.9236
Epoch: 20/40 Iteration: 5 Train loss: 0.025 Train accuracy: 0.9259
Epoch: 20/40 Iteration: 10 Train loss: 0.066 Train accuracy: 0.9262
Epoch: 20/40 Iteration: 15 Train loss: 0.062 Train accuracy: 0.9267
Epoch: 20/40 Iteration: 20 Train loss: 0.001 Train accuracy: 0.9271
Epoch: 20/40 Iteration: 25 Train loss: 0.002 Train accuracy: 0.9275
Epoch: 20/40 Iteration: 30 Train loss: 0.001 Train accuracy: 0.9280
Train accuracy (mean): 0.9269
Epoch: 21/40 Iteration: 5 Train loss: 0.079 Train accuracy: 0.9287
Epoch: 21/40 Iteration: 10 Train loss: 0.529 Train accuracy: 0.9290
Epoch: 21/40 Iteration: 15 Train loss: 0.011 Train accuracy: 0.9294
Epoch: 21/40 Iteration: 20 Train loss: 0.015 Train accuracy: 0.9298
Epoch: 21/40 Iteration: 25 Train loss: 0.014 Train accuracy: 0.9301
Epoch: 21/40 Iteration: 30 Train loss: 0.005 Train accuracy: 0.9306
Train accuracy (mean): 0.9296
Epoch: 22/40 Iteration: 5 Train loss: 0.004 Train accuracy: 0.9313
Epoch: 22/40 Iteration: 10 Tr

Epoch: 38/40 Iteration: 30 Train loss: 0.000 Train accuracy: 0.9566
Train accuracy (mean): 0.9562
Epoch: 39/40 Iteration: 5 Train loss: 0.000 Train accuracy: 0.9569
Epoch: 39/40 Iteration: 10 Train loss: 0.000 Train accuracy: 0.9571
Epoch: 39/40 Iteration: 15 Train loss: 0.000 Train accuracy: 0.9573
Epoch: 39/40 Iteration: 20 Train loss: 0.000 Train accuracy: 0.9574
Epoch: 39/40 Iteration: 25 Train loss: 0.000 Train accuracy: 0.9576
Epoch: 39/40 Iteration: 30 Train loss: 0.000 Train accuracy: 0.9578
Train accuracy (mean): 0.9574
Epoch: 40/40 Iteration: 5 Train loss: 0.000 Train accuracy: 0.9580
Epoch: 40/40 Iteration: 10 Train loss: 0.000 Train accuracy: 0.9582
Epoch: 40/40 Iteration: 15 Train loss: 0.000 Train accuracy: 0.9584
Epoch: 40/40 Iteration: 20 Train loss: 0.000 Train accuracy: 0.9585
Epoch: 40/40 Iteration: 25 Train loss: 0.000 Train accuracy: 0.9587
Epoch: 40/40 Iteration: 30 Train loss: 0.000 Train accuracy: 0.9588
Train accuracy (mean): 0.9584
duration: 250.5 sec


In [17]:
# this part calculates the number of trainable parameters

model = ResNet_model(learning_rate = learning_rate)
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())

    total_parameters = 0
    for variable in tf.trainable_variables():
        print(variable)
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print('shape of weight matrix: ',shape)
        #print(len(shape))
        variable_parameters = 1
        for dim in shape:
            #print(dim)
            variable_parameters *= dim.value
        print('number of trainable parameters: ',variable_parameters)
        print('------------------------------')
        total_parameters += variable_parameters
    print('total number of trainable parameters: ',total_parameters)

# this part calculates the number of all the  parameters
    total_parameters = 0
    for variable in tf.all_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    print('total number of parameters: ',total_parameters)

W0812 03:13:24.783797 16876 deprecation.py:323] From <ipython-input-17-6d8607952ead>:25: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


<tf.Variable 'conv1/kernel:0' shape=(7, 7, 3, 64) dtype=float32_ref>
shape of weight matrix:  (7, 7, 3, 64)
number of trainable parameters:  9408
------------------------------
<tf.Variable 'conv1/bias:0' shape=(64,) dtype=float32_ref>
shape of weight matrix:  (64,)
number of trainable parameters:  64
------------------------------
<tf.Variable 'bn_conv1/gamma:0' shape=(64,) dtype=float32_ref>
shape of weight matrix:  (64,)
number of trainable parameters:  64
------------------------------
<tf.Variable 'bn_conv1/beta:0' shape=(64,) dtype=float32_ref>
shape of weight matrix:  (64,)
number of trainable parameters:  64
------------------------------
<tf.Variable 'W1_convolutional/res2a_branch2a/kernel:0' shape=(1, 1, 64, 64) dtype=float32_ref>
shape of weight matrix:  (1, 1, 64, 64)
number of trainable parameters:  4096
------------------------------
<tf.Variable 'W1_convolutional/res2a_branch2a/bias:0' shape=(64,) dtype=float32_ref>
shape of weight matrix:  (64,)
number of trainable para

## Testing

In [18]:
tf.reset_default_graph()
model = ResNet_model(learning_rate = learning_rate)

test_acc = []
with tf.Session(config=config) as sess:
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    sess.run(tf.local_variables_initializer())
    
    iteration = 1
    #for (x, y) in get_batches(X_test, Y_test, batch_size):
    for (x, y) in random_mini_batches(X_test, Y_test, mini_batch_size = mini_batch_size, seed = 0):
        feed = {model.inputs_: x,
                   model.targets_: y,
                   model.training: False}


        batch_loss, batch_accuracy, batch_acc, batch_acc_op = sess.run([model.loss, model.accuracy,
                                                                       model.acc, model.acc_op],
                                 feed_dict=feed)
            
        test_acc.append(batch_acc_op)
        
        if iteration%1==0:
            print("Iteration: {}".format(iteration),
                      "Test loss: {:.3f}".format(batch_loss),
                      "Test accuracy: {:.4f}".format(batch_accuracy)
                     )
                     
                
        iteration +=1

    print("Test accuracy (mean): {:.4f} ".format(np.mean(test_acc)))

W0812 03:13:35.282984 16876 deprecation.py:323] From C:\Users\vgkortsas\AppData\Local\Continuum\anaconda3\envs\TF_practice\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Iteration: 1 Test loss: 0.013 Test accuracy: 1.0000
Iteration: 2 Test loss: 0.037 Test accuracy: 0.9688
Iteration: 3 Test loss: 0.003 Test accuracy: 1.0000
Iteration: 4 Test loss: 0.158 Test accuracy: 0.9583
Test accuracy (mean): 0.9893 
